In [10]:
import pandas as pd
import numpy as np 
from keras.layers import Dense
from keras.models import Sequential

from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv("https://excelr.s3.nl-ams.scw.cloud/DataScience-Assignments/updated-Folders/Neural-Network/gas_turbines.csv")

In [3]:
df.head()

,AT,AP,AH,AFDP,GTEP,TIT,TAT,TEY,CDP,CO,NOX
0,6.8594,1007.9,96.799,3.5000,19.663,1059.2,550.00,114.70,10.605,3.1547,82.722
1,6.7850,1008.4,97.118,3.4998,19.728,1059.3,550.00,114.72,10.598,3.2363,82.776
2,6.8977,1008.8,95.939,3.4824,19.779,1059.4,549.87,114.71,10.601,3.2012,82.468
3,7.0569,1009.2,95.249,3.4805,19.792,1059.6,549.99,114.72,10.606,3.1923,82.670
4,7.3978,1009.7,95.150,3.4976,19.765,1059.7,549.98,114.72,10.612,3.2484,82.311


In [4]:
df.shape

(15039, 11)

In [5]:
x = df.drop(['TEY'],axis = 1)
y = df['TEY']

In [6]:
x.head()

,AT,AP,AH,AFDP,GTEP,TIT,TAT,CDP,CO,NOX
0,6.8594,1007.9,96.799,3.5000,19.663,1059.2,550.00,10.605,3.1547,82.722
1,6.7850,1008.4,97.118,3.4998,19.728,1059.3,550.00,10.598,3.2363,82.776
2,6.8977,1008.8,95.939,3.4824,19.779,1059.4,549.87,10.601,3.2012,82.468
3,7.0569,1009.2,95.249,3.4805,19.792,1059.6,549.99,10.606,3.1923,82.670
4,7.3978,1009.7,95.150,3.4976,19.765,1059.7,549.98,10.612,3.2484,82.311


In [7]:
y.head()

0    114.70
1    114.72
2    114.71
3    114.72
4    114.72
Name: TEY, dtype: float64

In [11]:
std = StandardScaler()
x_std = std.fit_transform(x)

In [12]:
x_std

array([[-1.4397781 , -0.82664395,  1.28143632, ..., -1.35733078,
         0.53201195,  1.3878449 ],
       [-1.44960109, -0.74864748,  1.30456402, ..., -1.36367619,
         0.56873344,  1.39300237],
       [-1.43472138, -0.68625031,  1.21908576, ..., -1.36095673,
         0.5529378 ,  1.36358566],
       ...,
       [-1.38626659, -1.07623263,  1.47697056, ..., -1.46792219,
         2.69592467,  2.17006209],
       [-1.42042259, -0.99823616,  1.44159024, ..., -1.42259784,
         1.9246834 ,  2.391165  ],
       [-1.43073409, -0.93583899,  1.33465179, ..., -1.37727349,
         1.35415028,  2.32153907]])

In [13]:
x.describe()

,AT,AP,AH,AFDP,GTEP,TIT,TAT,CDP,CO,NOX
count,15039.000000,15039.00000,15039.000000,15039.000000,15039.000000,15039.000000,15039.000000,15039.000000,15039.000000,15039.000000
mean,17.764381,1013.19924,79.124174,4.200294,25.419061,1083.798770,545.396183,12.102353,1.972499,68.190934
std,7.574323,6.41076,13.793439,0.760197,4.173916,16.527806,7.866803,1.103196,2.222206,10.470586
min,0.522300,985.85000,30.344000,2.087400,17.878000,1000.800000,512.450000,9.904400,0.000388,27.765000
25%,11.408000,1008.90000,69.750000,3.723900,23.294000,1079.600000,542.170000,11.622000,0.858055,61.303500
50%,18.186000,1012.80000,82.266000,4.186200,25.082000,1088.700000,549.890000,12.025000,1.390200,66.601000
75%,23.862500,1016.90000,90.043500,4.550900,27.184000,1096.000000,550.060000,12.578000,2.160400,73.935500
max,34.929000,1034.20000,100.200000,7.610600,37.402000,1100.800000,550.610000,15.081000,44.103000,119.890000


#### Hyper tuning of parameters

In [14]:
from sklearn.model_selection import GridSearchCV,KFold
from keras.wrappers.scikit_learn import KerasRegressor      # As the task is Regression
from tensorflow.keras.optimizers import Adam


#### Creating a framework

In [15]:
def create_model():
    model = Sequential()
    model.add(Dense(8,input_dim = 10,kernel_initializer = 'uniform', activation = 'relu'))
    model.add(Dense(5,kernel_initializer = 'uniform', activation = 'relu'))
    model.add(Dense(1,kernel_initializer = 'uniform', activation = 'sigmoid'))
    
    adam = Adam(lr = 0.01)
    model.compile(loss = 'mean_squared_error', optimizer = 'adam', metrics = ['accuracy'])
    return model

##### Creating the model

##### Tuning for batch size and epochs

In [17]:
model = KerasRegressor(build_fn = create_model, verbose = 0)

batch_size = [60,80,100]
epochs = [20,40,60]

param_grid = dict(batch_size = batch_size, epochs = epochs)

grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(), verbose = 10)
grid.fit(x_std,y)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] batch_size=60, epochs=20 ........................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


[CV] ....... batch_size=60, epochs=20, score=-18533.217, total=   5.7s
[CV] batch_size=60, epochs=20 ........................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    5.7s remaining:    0.0s


[CV] ....... batch_size=60, epochs=20, score=-16715.756, total=   5.8s
[CV] batch_size=60, epochs=20 ........................................


...

[CV] ....... batch_size=60, epochs=40, score=-18239.477, total=  10.8s
[CV] batch_size=60, epochs=40 ........................................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  1.1min remaining:    0.0s


[CV] ....... batch_size=60, epochs=40, score=-17266.109, total=  10.8s
[CV] batch_size=60, epochs=60 ........................................
[CV] ....... batch_size=60, epochs=60, score=-18533.189, total=  12.1s
[CV] batch_size=60, epochs=60 ........................................
[CV] ....... batch_size=60, epochs=60, score=-16715.730, total=  12.2s
[CV] batch_size=60, epochs=60 ........................................
[CV] ....... batch_size=60, epochs=60, score=-19193.916, total=  21.1s
[CV] batch_size=60, epochs=60 ........................................
[CV] ....... batch_size=60, epochs=60, score=-18239.477, total=  12.1s
[CV] batch_size=60, epochs=60 ........................................
[CV] ....... batch_size=60, epochs=60, score=-17266.109, total=  12.1s
[CV] batch_size=80, epochs=20 ........................................
[CV] ....... batch_size=80, epochs=20, score=-18533.244, total=   5.7s
[CV] batch_size=80, epochs=20 ........................................
[CV] .

[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:  5.8min finished


GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             error_score=nan,
             estimator=<keras.wrappers.scikit_learn.KerasRegressor object at 0x7f35103c6290>,
             iid='deprecated', n_jobs=None,
             param_grid={'batch_size': [60, 80, 100], 'epochs': [20, 40, 60]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=10)

In [18]:
print('Best: {} with: {}'.format(grid.best_score_ , grid.best_params_))

means = grid.cv_results_['mean_test_score']
std = grid.cv_results_['std_test_score']
params = grid.cv_results_['params']

for mean,stdev,param in zip(means,std,params):
       print('{} {} with : {}'.format(mean,stdev,param))
   

Best: -17989.6828125 with: {'batch_size': 80, 'epochs': 60}
-17989.703515625 889.3276225973397 with : {'batch_size': 60, 'epochs': 20}
-17990.115625 889.5883592729041 with : {'batch_size': 60, 'epochs': 40}
-17989.684375 889.3241351769005 with : {'batch_size': 60, 'epochs': 60}
-17989.71640625 889.3200424295077 with : {'batch_size': 80, 'epochs': 20}
-17989.691796875 889.3281063027836 with : {'batch_size': 80, 'epochs': 40}
-17989.6828125 889.3247768605675 with : {'batch_size': 80, 'epochs': 60}
-17989.7265625 889.3417704453185 with : {'batch_size': 100, 'epochs': 20}
-17989.691796875 889.3232276598369 with : {'batch_size': 100, 'epochs': 40}
-17989.688671875 889.3279220605541 with : {'batch_size': 100, 'epochs': 60}


##### Tuning for dropout rate and learning rate

In [19]:
from keras.layers import Dropout

def create_model(learning_rate,dropout_rate):
    model = Sequential()
    model.add(Dense(8,input_dim = 10,kernel_initializer = 'uniform', activation = 'relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(5,kernel_initializer = 'uniform', activation = 'relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1,kernel_initializer = 'uniform', activation = 'sigmoid'))
    
    adam = Adam(lr = learning_rate)
    model.compile(loss = 'mean_squared_error', optimizer = 'adam', metrics = ['accuracy'])
    return model

model = KerasRegressor(build_fn = create_model,batch_size = 80, epochs = 70, verbose = 0)

dropout_rate = [0.1,0.2,0.5]
learning_rate = [0.1,0.01,0.001]

param_grid = dict(dropout_rate = dropout_rate, learning_rate = learning_rate)

grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(n_splits = 3), verbose = 10)
grid.fit(x_std,y)

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] dropout_rate=0.1, learning_rate=0.1 .............................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


[CV]  dropout_rate=0.1, learning_rate=0.1, score=-17761.498, total=  10.9s
[CV] dropout_rate=0.1, learning_rate=0.1 .............................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   10.9s remaining:    0.0s


[CV]  dropout_rate=0.1, learning_rate=0.1, score=-18855.252, total=   9.6s
[CV] dropout_rate=0.1, learning_rate=0.1 .............................


...

[CV]  dropout_rate=0.1, learning_rate=0.001, score=-17352.438, total=   9.7s
[CV] dropout_rate=0.2, learning_rate=0.1 .............................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  1.5min remaining:    0.0s


[CV]  dropout_rate=0.2, learning_rate=0.1, score=-17761.500, total=  10.9s
[CV] dropout_rate=0.2, learning_rate=0.1 .............................
[CV]  dropout_rate=0.2, learning_rate=0.1, score=-18855.252, total=   9.5s
[CV] dropout_rate=0.2, learning_rate=0.1 .............................
[CV]  dropout_rate=0.2, learning_rate=0.1, score=-17352.438, total=   9.6s
[CV] dropout_rate=0.2, learning_rate=0.01 ............................
[CV]  dropout_rate=0.2, learning_rate=0.01, score=-17761.498, total=  10.0s
[CV] dropout_rate=0.2, learning_rate=0.01 ............................
[CV]  dropout_rate=0.2, learning_rate=0.01, score=-18855.252, total=   9.7s
[CV] dropout_rate=0.2, learning_rate=0.01 ............................
[CV]  dropout_rate=0.2, learning_rate=0.01, score=-17352.438, total=  10.9s
[CV] dropout_rate=0.2, learning_rate=0.001 ...........................
[CV]  dropout_rate=0.2, learning_rate=0.001, score=-17761.498, total=   9.6s
[CV] dropout_rate=0.2, learning_rate=0.001 .

[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:  4.6min finished


GridSearchCV(cv=KFold(n_splits=3, random_state=None, shuffle=False),
             error_score=nan,
             estimator=<keras.wrappers.scikit_learn.KerasRegressor object at 0x7f350f7549d0>,
             iid='deprecated', n_jobs=None,
             param_grid={'dropout_rate': [0.1, 0.2, 0.5],
                         'learning_rate': [0.1, 0.01, 0.001]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=10)

In [20]:
print('Best: {} with: {}'.format(grid.best_score_ , grid.best_params_))

means = grid.cv_results_['mean_test_score']
std = grid.cv_results_['std_test_score']
params = grid.cv_results_['params']

for mean,stdev,param in zip(means,std,params):
       print('{} {} with : {}'.format(mean,stdev,param))
   

Best: -17989.729166666668 with: {'dropout_rate': 0.1, 'learning_rate': 0.01}
-17990.229166666668 633.8899566230544 with : {'dropout_rate': 0.1, 'learning_rate': 0.1}
-17989.729166666668 634.3920465881051 with : {'dropout_rate': 0.1, 'learning_rate': 0.01}
-17989.729166666668 634.3920465881051 with : {'dropout_rate': 0.1, 'learning_rate': 0.001}
-17989.729817708332 634.3918123676842 with : {'dropout_rate': 0.2, 'learning_rate': 0.1}
-17989.729166666668 634.3920465881051 with : {'dropout_rate': 0.2, 'learning_rate': 0.01}
-17989.729166666668 634.3920465881051 with : {'dropout_rate': 0.2, 'learning_rate': 0.001}
-17989.729166666668 634.3920465881051 with : {'dropout_rate': 0.5, 'learning_rate': 0.1}
-17989.729166666668 634.3920465881051 with : {'dropout_rate': 0.5, 'learning_rate': 0.01}
-17989.729166666668 634.3920465881051 with : {'dropout_rate': 0.5, 'learning_rate': 0.001}


##### Tuning for Activation Function and kernel initializer

In [21]:
from keras.layers import Dropout

def create_model(activation_function,init):
    model = Sequential()
    model.add(Dense(8,input_dim = 10,kernel_initializer = init, activation = activation_function))
    model.add(Dropout(0.1))
    model.add(Dense(5,kernel_initializer = init, activation = activation_function))
    model.add(Dropout(0.1))
    model.add(Dense(1,kernel_initializer = 'uniform', activation = 'sigmoid'))
    
    adam = Adam(lr = 0.001)
    model.compile(loss = 'mean_squared_error', optimizer = 'adam', metrics = ['accuracy'])
    return model

model = KerasRegressor(build_fn = create_model,batch_size = 80, epochs = 70, verbose = 0)

activation_function = ['relu','maxout','elu','tanh','sigmoid']
init = ['uniform','normal','zero']

param_grid = dict(activation_function = activation_function, init = init)

grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(n_splits = 3), verbose = 10)
grid.fit(x_std,y)

Fitting 3 folds for each of 15 candidates, totalling 45 fits
[CV] activation_function=relu, init=uniform ..........................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


[CV]  activation_function=relu, init=uniform, score=-17761.504, total=  10.9s
[CV] activation_function=relu, init=uniform ..........................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   10.9s remaining:    0.0s


[CV]  activation_function=relu, init=uniform, score=-18855.252, total=  10.9s
[CV] activation_function=relu, init=uniform ..........................


...

[CV]  activation_function=relu, init=zero, score=-17353.938, total=  11.0s
[CV] activation_function=maxout, init=uniform ........................
[CV]  activation_function=maxout, init=uniform, score=nan, total=   0.0s
[CV] activation_function=maxout, init=uniform ........................
[CV]  activation_function=maxout, init=uniform, score=nan, total=   0.0s
[CV] activation_function=maxout, init=uniform ........................
[CV]  activation_function=maxout, init=uniform, score=nan, total=   0.0s
[CV] activation_function=maxout, init=normal .........................
[CV]  activation_function=maxout, init=normal, score=nan, total=   0.0s
[CV] activation_function=maxout, init=normal .........................
[CV]  activation_function=maxout, init=normal, score=nan, total=   0.0s
[CV] activation_function=maxout, init=normal .........................
[CV]  activation_function=maxout, init=normal, score=nan, total=   0.0s
[CV] activation_function=maxout, init=zero .....................

[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  1.6min remaining:    0.0s
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Unknown activation function: maxout. Please ensure this object is passed to the `custom_objects` argument. See https://www.tensorflow.org/guide/keras/save_and_serialize#registering_the_custom_object for details.

  FitFailedWarning)


[CV]  activation_function=elu, init=uniform, score=-17761.498, total=   9.5s
[CV] activation_function=elu, init=uniform ...........................
[CV]  activation_function=elu, init=uniform, score=-18855.252, total=  10.9s
[CV] activation_function=elu, init=uniform ...........................
[CV]  activation_function=elu, init=uniform, score=-17352.438, total=   9.6s
[CV] activation_function=elu, init=normal ............................
[CV]  activation_function=elu, init=normal, score=-17761.498, total=   9.5s
[CV] activation_function=elu, init=normal ............................
[CV]  activation_function=elu, init=normal, score=-18855.252, total=  10.9s
[CV] activation_function=elu, init=normal ............................
[CV]  activation_function=elu, init=normal, score=-17352.438, total=   9.9s
[CV] activation_function=elu, init=zero ..............................
[CV]  activation_function=elu, init=zero, score=-17761.498, total=   9.7s
[CV] activation_function=elu, init=zero .

[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:  6.2min finished


GridSearchCV(cv=KFold(n_splits=3, random_state=None, shuffle=False),
             error_score=nan,
             estimator=<keras.wrappers.scikit_learn.KerasRegressor object at 0x7f350f43f850>,
             iid='deprecated', n_jobs=None,
             param_grid={'activation_function': ['relu', 'maxout', 'elu',
                                                 'tanh', 'sigmoid'],
                         'init': ['uniform', 'normal', 'zero']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=10)

In [ ]:
print('Best: {} with: {}'.format(grid.best_score_ , grid.best_params_))

means = grid.cv_results_['mean_test_score']
std = grid.cv_results_['std_test_score']
params = grid.cv_results_['params']

for mean,stdev,param in zip(means,std,params):
       print('{} {} with : {}'.format(mean,stdev,param))
   

Best: -17989.729166666668 with: {'activation_function': 'relu', 'init': 'normal'}
-17989.729817708332 634.3918123676842 with : {'activation_function': 'relu', 'init': 'uniform'}
-17989.729166666668 634.3920465881051 with : {'activation_function': 'relu', 'init': 'normal'}
-17991.259765625 634.421446741094 with : {'activation_function': 'relu', 'init': 'zero'}
nan nan with : {'activation_function': 'maxout', 'init': 'uniform'}
nan nan with : {'activation_function': 'maxout', 'init': 'normal'}
nan nan with : {'activation_function': 'maxout', 'init': 'zero'}
-17989.729166666668 634.3920465881051 with : {'activation_function': 'elu', 'init': 'uniform'}
-17989.729166666668 634.3920465881051 with : {'activation_function': 'elu', 'init': 'normal'}
-17989.73046875 634.3911583497184 with : {'activation_function': 'elu', 'init': 'zero'}
-17989.739583333332 634.3925150326963 with : {'activation_function': 'tanh', 'init': 'uniform'}
-17989.739583333332 634.3925150326963 with : {'activation_functio

##### checking number of neurons

In [ ]:
def create_model(neuron1,neuron2):
    model = Sequential()
    model.add(Dense(8,input_dim = 10,kernel_initializer = 'normal', activation = 'relu'))
    model.add(Dropout(0.1))
    model.add(Dense(5,kernel_initializer = 'normal', activation = 'relu'))
    model.add(Dropout(0.1))
    model.add(Dense(1,kernel_initializer = 'normal', activation = 'relu'))
    
    adam = Adam(lr = 0.001)
    model.compile(loss = 'mean_squared_error', optimizer = 'adam', metrics = ['accuracy'])
    return model

model = KerasRegressor(build_fn = create_model,batch_size = 80, epochs = 70, verbose = 0)

neuron1 = [5,8,10]
neuron2 = [2,3,5]

param_grid = dict(neuron1 = neuron1, neuron2 = neuron2)

grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(), verbose = 10)
grid.fit(x_std,y)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5; 1/9] START neuron1=5, neuron2=2........................................
[CV 1/5; 1/9] END ......................neuron1=5, neuron2=2; total time=  15.2s
[CV 2/5; 1/9] START neuron1=5, neuron2=2........................................
[CV 2/5; 1/9] END ......................neuron1=5, neuron2=2; total time=  16.7s
[CV 3/5; 1/9] START neuron1=5, neuron2=2........................................
[CV 3/5; 1/9] END ......................neuron1=5, neuron2=2; total time=  16.8s
[CV 4/5; 1/9] START neuron1=5, neuron2=2........................................
[CV 4/5; 1/9] END ......................neuron1=5, neuron2=2; total time=  24.5s
[CV 5/5; 1/9] START neuron1=5, neuron2=2........................................
[CV 5/5; 1/9] END ......................neuron1=5, neuron2=2; total time=  15.3s
[CV 1/5; 2/9] START neuron1=5, neuron2=3........................................
[CV 1/5; 2/9] END ......................neuron1=5

GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             estimator=<keras.wrappers.scikit_learn.KerasRegressor object at 0x0000025B517CE6A0>,
             param_grid={'neuron1': [5, 8, 10], 'neuron2': [2, 3, 5]},
             verbose=10)

In [22]:
print('Best: {} with: {}'.format(grid.best_score_ , grid.best_params_))

means = grid.cv_results_['mean_test_score']
std = grid.cv_results_['std_test_score']
params = grid.cv_results_['params']

for mean,stdev,param in zip(means,std,params):
       print('{} {} with : {}'.format(mean,stdev,param))
   

Best: -17989.729166666668 with: {'activation_function': 'relu', 'init': 'normal'}
-17989.731119791668 634.3913439305916 with : {'activation_function': 'relu', 'init': 'uniform'}
-17989.729166666668 634.3920465881051 with : {'activation_function': 'relu', 'init': 'normal'}
-17991.259765625 634.421446741094 with : {'activation_function': 'relu', 'init': 'zero'}
nan nan with : {'activation_function': 'maxout', 'init': 'uniform'}
nan nan with : {'activation_function': 'maxout', 'init': 'normal'}
nan nan with : {'activation_function': 'maxout', 'init': 'zero'}
-17989.729166666668 634.3920465881051 with : {'activation_function': 'elu', 'init': 'uniform'}
-17989.729166666668 634.3920465881051 with : {'activation_function': 'elu', 'init': 'normal'}
-17989.731119791668 634.3900845389222 with : {'activation_function': 'elu', 'init': 'zero'}
-17989.738932291668 634.3916267936291 with : {'activation_function': 'tanh', 'init': 'uniform'}
-17989.738932291668 634.3916267936291 with : {'activation_fun

In [23]:
from sklearn.model_selection import train_test_split

In [24]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2, random_state = 101)

In [25]:
std1 = StandardScaler()
x_train_std = std1.fit_transform(x_train)
x_test_std = std1.fit_transform(x_test)

##### Building the final model with optimum values achieved by grid search cv

In [26]:
from tensorflow.keras.optimizers import Adam

model = Sequential()
model.add(Dense(5,input_dim = 10, kernel_initializer = 'uniform', activation = 'relu'))
model.add(Dense(3,kernel_initializer = 'uniform', activation = 'relu'))
model.add(Dense(1,kernel_initializer = 'uniform', activation = 'relu'))

adam = Adam(lr = 0.001)    
model.compile(optimizer='adam',loss='mse')
model.fit(x_train_std,y_train, epochs = 70, batch_size = 80)


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/70
151/151 [==============================] - 1s 1ms/step - loss: 18200.1758
Epoch 2/70
151/151 [==============================] - 0s 1ms/step - loss: 17881.9512
Epoch 3/70
151/151 [==============================] - 0s 1ms/step - loss: 16791.8379
Epoch 4/70
151/151 [==============================] - 0s 1ms/step - loss: 14717.6328
Epoch 5/70
151/151 [==============================] - 0s 1ms/step - loss: 12039.5830
Epoch 6/70
151/151 [==============================] - 0s 1ms/step - loss: 9399.9326
Epoch 7/70
151/151 [==============================] - 0s 1ms/step - loss: 7081.8022
Epoch 8/70
151/151 [==============================] - 0s 1ms/step - loss: 5067.8765
Epoch 9/70
151/151 [==============================] - 0s 1ms/step - loss: 3392.7983
Epoch 10/70
151/151 [==============================] - 0s 1ms/step - loss: 2159.0298
Epoch 11/70
151/151 [==============================] - 0s 1ms/step - loss: 1388.5795
Epoch 12/70
151/151 [==============================] - 0s 1ms/step - 

In [27]:
score = model.evaluate(x_test_std,y_test)

94/94 [==============================] - 0s 959us/step - loss: 5.8415
